# 한글-Claude-v2 Model: Question and answers with Bedrock
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.




QA(Question Answering)는 자연어로 제기된 사실적 질문에 대한 답변을 추출하는 중요한 작업입니다. 일반적으로 QA 시스템은 구조화되거나 구조화되지 않은 데이터가 포함된 지식 기반에 대한 쿼리를 처리하고 정확한 정보로 응답을 생성합니다. 높은 정확도를 보장하는 것은 특히 엔터프라이즈 사용 사례에서 유용하고 신뢰할 수 있으며 신뢰할 수 있는 질문 응답 시스템을 개발하는 데 핵심입니다.

Titan 및 Claude와 같은 생성 AI 모델은 확률 분포를 사용하여 질문에 대한 응답을 생성합니다. 이 모델은 방대한 양의 텍스트 데이터에 대해 훈련되어 시퀀스에서 다음에 오는 내용이나 특정 단어 뒤에 어떤 단어가 뒤따를지 예측할 수 있습니다. 그러나 이러한 모델은 데이터에 항상 어느 정도의 불확실성이 있기 때문에 모든 질문에 대해 정확하거나 결정적인 답변을 제공할 수 없습니다. 기업은 도메인별 독점 데이터를 쿼리하고 정보를 사용하여 질문에 답해야 하며 일반적으로 모델이 훈련되지 않은 데이터를 사용해야 합니다.

이 모듈에서는 Bedrock Titan 모델을 사용하여 쿼리에 대한 정보 응답을 제공하는 방법을 보여줍니다.

이 예제에서는 컨텍스트 없이 모델을 실행한 다음 수동으로 컨텍스트를 제공하려고 합니다. 여기서는 'RAG' 확대가 발생하지 않습니다. 이 접근 방식은 짧은 문서 또는 단일 톤 애플리케이션에서 작동하며 모델에 전송된 프롬프트에 모두 맞지 않는 대규모 엔터프라이즈 문서가 있을 수 있는 엔터프라이즈 수준 질문 응답으로 확장되지 않을 수 있습니다.

### 도전
- 모델이 질문에 대한 사실적인 답변을 반환하도록 하는 방법

### 제안
위의 과제에 대해 본 노트북에서는 다음과 같은 전략을 제안합니다.
#### 문서 준비
질문에 답하기 전에 문서를 처리하고 문서 저장소 색인에 저장해야 합니다.
- 여기에서 전체 관련 컨텍스트와 함께 요청을 모델에 보내고 응답을 기대합니다.

### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되었는지 확인해야 합니다.

## 1. Bedrock Client 생성

In [3]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.312
opensearch-py                        2.3.2


In [4]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 2. 섹션 1: 모델 지식과의 Q&A
이 섹션에서는 훈련 단계에서 얻은 지식을 기반으로 질문에 답하기 위해 Bedrock 서비스에서 제공하는 모델을 사용하려고 합니다.

이 노트북에서는 Amazon Bedrock 클라이언트의 `invoke_model()` 메서드를 사용합니다. 이 방법을 사용하는 데 필요한 필수 매개변수는 Amazon Bedrock 모델 ARN을 나타내는 'modelId'와 작업 프롬프트인 'body'입니다. 선택한 기본 모델 공급자에 따라 `body` 프롬프트가 변경됩니다. 아래에서 자세히 살펴보겠습니다.
```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

### 시나리오

모델에게 타이어 교체 정보 제공을 요청하는 상황을 모델링 하려고 합니다. 먼저 학습 데이터를 기반으로 모델에 요청하여 자동차의 특정 제조업체 및 모델에 대한 답변을 제공합니다. 이 기술을 '제로샷'이라고 합니다. 우리는 곧 모델이 우리의 특정 자동차와 관련된 답변을 반환하는 것처럼 보이지만 실제로는 환각을 일으키고 있음을 깨닫게 될 것입니다. 우리가 그것을 알아낼 수 있는 이유는 우리가 가짜 자동차를 통과하고 거의 비슷한 시나리오를 받고 대답하기 때문입니다.

이 상황은 자동차의 특정 제조업체 및 모델에 대한 추가 데이터로 모델의 학습을 보강해야 함을 의미하며, 그러면 모델이 매우 구체적인 답변을 반환합니다. 이 노트북에서는 데이터를 보강하기 위해 외부 소스를 사용하지 않고 RAG 기반 보강 시스템이 작동하는 방식을 시뮬레이션합니다.

최종 테스트를 실행하기 위해 우리는 특정 차량에 대한 타이어 변경 작동 방식을 설명하는 설명서의 전체 세부 섹션을 제공한 다음 모델에서 선별된 응답을 다시 얻기 위해 테스트합니다.

### 작업

프로세스를 시작하려면 Bedrock에서 제공하는 모델 중 하나를 선택합니다. 이 사용 사례에서는 Claude-v2을 선택합니다. 이러한 모델은 자동차에 대한 일반적인 질문에 답할 수 있습니다.

예를 들어 Claude-v2 모델에게 Audi의 펑크 난 타이어를 교체하는 방법을 알려달라고 요청합니다.

## 3. Audi A8 펑크 타이어 수리 질문

In [7]:
prompt_data = """당신은 도움이 되는 조수입니다. 간결한 방식으로 질문에 답하십시오. 당신이 확실하지 않은 경우
대답은 '확실하지 않습니다' 라고 말하세요.

질문: Audi A8에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
답변:"""

# Claude Prompt
body = json.dumps({
    "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
    "max_tokens_to_sample": 500,
    "temperature": 0.3,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]            
    }) 

#### JSON 본문을 전달하는 모델을 호출하여 응답을 생성해 보겠습니다.

In [8]:
# body = json.dumps({"inputText": prompt_data, "textGenerationConfig": parameters})
modelId = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

def inference_model(body, modelId, accept, contentType):
    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    answer = response_body.get("completion")

    print_ww("prompt_data: \n", prompt_data)
    print("### Answer from Claude-v2 ####")
    print_ww(answer.strip())
    
inference_model(body, modelId, accept, contentType)    

prompt_data:
 당신은 도움이 되는 조수입니다. 간결한 방식으로 질문에 답하십시오. 당신이 확실하지 않은 경우
대답은 '확실하지 않습니다' 라고 말하세요.

질문: Audi A8에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
답변:
### Answer from Claude-v2 ####
Audi A8의 타이어 펑크를 고치는 방법은 다음과 같습니다:

1. 안전한 장소에 차를 세우세요. 도로에서 떨어진 곳이 좋습니다.

2. 경고등을 켜고 고장 삼각대를 세우세요.

3. 잭과 휠 렌치를 꺼내세요. 잭은 차량 트렁크에 있습니다.

4. 펑크난 타이어 주변의 너트를 풉니다. 차가 전복되지 않도록 주의하세요.

5. 잭을 사용하여 차를 들어 올리고 펑크난 타이어를 분리합니다.

6. 스페어 타이어를 꺼내서 펑크난 타이어 자리에 넣습니다.

7. 너트를 조이고 고정합니다. 차량이 안정적인지 확인하세요.

8. 잭을 내리고 도구들을 정리합니다.

9. 가능한 빨리 타이어 수리점에서 펑크난 타이어를 수리하세요.

이렇게 하면 Audi A8의 펑크난 타이어를 안전하게 교체할 수 있습니다. 제가 확실하지 않은 부분이 있다면 말씀해 주세요.


모델은 자동차 펑크 난 타이어를 교체하는 과정을 간략히 설명하는 답변을 제공할 수 있지만 동일한 설명이 모든 자동차에 유효할 수 있습니다. 불행히도 이것은 스페어 타이어가 없는 Audi A8에 대한 정답이 아닙니다. 이는 모델이 자동차 타이어 교체에 대한 지침이 포함된 데이터로 학습되었기 때문입니다.

이 문제의 또 다른 예는 Amazon Tirana와 같이 완전히 가짜인 자동차 브랜드 및 모델에 대해 동일한 질문을 시도하여 볼 수 있습니다.

## 4. 가상의 자동차 (Amazon Tirana) 펑크 타이어 질문 <-- 환각 가능성

In [9]:
prompt_data = "Amazon Tirana에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?"
body = json.dumps({
    "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
    "max_tokens_to_sample": 500,
    "temperature": 0.3,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]            
    }) 

inference_model(body, modelId, accept, contentType)    

prompt_data:
 Amazon Tirana에서 펑크 난 타이어를 어떻게 고칠 수 있습니까?
### Answer from Claude-v2 ####
제가 아는 한 알바니아의 수도 티라나에서 펑크난 타이어를 고치는 몇 가지 방법이 있습니다:

- 타이어 수리점 방문 - 티라나에는 타이어 수리와 교체를 전문으로 하는 많은 가게가 있습니다. 가까운 곳을 찾아서 펑크난 타이어를 수리하거나 교체하실 수 있습니다.

- 도로 조력 서비스 이용 - 알바니아의 자동차 협회들은 도로에서 고장난 차량을 돕는 서비스를 제공합니다. 그들에게 연락하여 펑크난 타이어를 현장에서 수리하거나 교체해줄 수
있습니다.

- 모바일 타이어 수리업체 이용 - 티라나에는 차량이 있는 곳으로 직접 방문하여 타이어를 수리 및 교체해주는 모바일 서비스도 있습니다. 앱이나 전화로 요청하면 편리하게 이용할 수
있습니다.

- 스페어 타이어로 교체 - 스페어 타이어가 있다면 직접 교체할 수도 있습니다. 그러나 영구적 해결책은 아니므로 가능한 빨리 타이어 전문점에서 수리하거나 새 타이어를 구매하는 것이
좋습니다.

펑크난 타이어는 위험하므로 가능한 빨리 안전하게 수리하는 것이 중요합니다.


## 5. 환각을 억제하기 위한 Context 제공

### Context 제공

보시다시피 모델이 제공하는 대답은 그럴듯합니다. 모델은 _환각_할 수 있습니다.

이 문제를 어떻게 해결하고 내 자동차 모델에 유효한 특정 지침에 따라 모델이 답변을 제공하도록 할 수 있습니까?

2020년 Facebook의 연구에 따르면 LLM 지식은 프롬프트의 일부로 추가 지식 기반을 제공함으로써 즉석에서 보강될 수 있습니다. 이 접근 방식을 RAG(Retrieval Augmented Generation)라고 합니다.

이를 사용하여 애플리케이션을 개선하는 방법을 살펴보겠습니다.

다음은 Audi A8의 설명서를 발췌한 것입니다(실제로는 실제 설명서가 아니지만 그렇다고 가정해 보겠습니다). 이 문서는 또한 Claude-v2 의 프롬프트에 완전히 들어갈 만큼 편리하게 짧습니다.

```
타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠 수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은 속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함 안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌리티 키트에 접근하려면 러기지 영역의 라이닝을 들어 올리십시오.
    타이어 이동성 키트와 함께 제공된 지침에 따라 타이어의 펑크를 밀봉하십시오.
    키트를 사용한 후에는 원래 위치에 안전하게 다시 넣어야 합니다.
    사용한 실란트 병의 폐기 및 교체에 대한 지원은 Rivesla 또는 적절한 서비스에 문의하십시오.

타이어 모빌리티 키트는 임시 솔루션이며 최대 10분 또는 8km(먼저 도래하는 것) 동안 최대 80km/h의 속도로 운전할 수 있도록 설계되었습니다. 펑크 난 타이어는 가능한 한 빨리 교체하거나 전문가에게 수리를 맡기는 것이 좋습니다.
```

 
다음으로 이 텍스트를 원래 질문과 함께 프롬프트에 "삽입"합니다. 또한 프롬프트는 컨텍스트로 제공된 정보만 보도록 모델에 힌트를 주는 방식으로 빌드됩니다.

In [10]:
context = """타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠 수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은 속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함 안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌리티 키트에 접근하려면 러기지 영역의 라이닝을 들어 올리십시오.
    타이어 이동성 키트와 함께 제공된 지침에 따라 타이어의 펑크를 밀봉하십시오.
    키트를 사용한 후에는 원래 위치에 안전하게 다시 넣어야 합니다.
    사용한 실란트 병의 폐기 및 교체에 대한 지원은 Rivesla 또는 적절한 서비스에 문의하십시오.

타이어 모빌리티 키트는 임시 솔루션이며 최대 10분 또는 8km(먼저 도래하는 것) 동안 최대 80km/h의 속도로 운전할 수 있도록 설계되었습니다. 펑크 난 타이어는 가능한 한 빨리 교체하거나 전문가에게 수리를 맡기는 것이 좋습니다.
"""

## 6. Context 를 제공하여 Claude-v2 에 질문

#### 전체 발췌문을 질문과 함께 모델에 전달해 보겠습니다.

In [11]:
question = "Audi A8의 펑크 난 타이어를 어떻게 고칠 수 있습니까?"
prompt_data = f"""## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
{context}
#

Question: {question}
"""

##### Invoke the model via boto3 to generate the response

In [12]:
print("prompt_data: \n", prompt_data)

prompt_data: 
 ## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠 수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은 속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함 안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌

In [13]:
body = json.dumps({
    "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
    "max_tokens_to_sample": 500,
    "temperature": 0.3,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]            
    }) 

inference_model(body, modelId, accept, contentType)

prompt_data:
 ## 사이에 제공된 정보만을 기준으로 질문에 답하고 단계별 안내를 제공합니다.
#
타이어 및 타이어 공기압:

타이어는 검은색 고무로 만들어졌으며 차량의 바퀴에 장착됩니다. 주행, 코너링 및 제동에 필요한 그립을 제공합니다. 타이어 공기압과 타이어 마모는 차량의 성능과 핸들링에 영향을 미칠
수 있으므로 고려해야 할 두 가지 중요한 요소입니다.

권장 타이어 공기압을 찾을 수 있는 곳:

차량의 운전석 쪽 B-필러에 있는 공기 주입 라벨에서 권장 타이어 공기압 사양을 확인할 수 있습니다. 또는 차량 설명서에서 이 정보를 참조할 수 있습니다. 권장 타이어 공기압은
속도와 탑승자 수 또는 차량의 최대 적재량에 따라 달라질 수 있습니다.

타이어 재공기:

타이어 공기압을 점검할 때는 타이어가 차가울 때 확인하는 것이 중요합니다. 이는 타이어가 주변 온도와 동일한 온도가 되도록 차량을 최소 3시간 동안 그대로 두는 것을 의미합니다.

타이어를 다시 채우려면:

    차량의 권장 타이어 공기압을 확인하십시오.
    공기 펌프에 제공된 지침을 따르고 타이어에 올바른 공기압을 주입하십시오.
    차량의 중앙 화면에서 "차량 상태" 앱을 엽니다.
    "타이어 공기압" 탭으로 이동합니다.
    "압력 보정" 옵션을 누르고 조치를 확인하십시오.
    타이어 공기압을 보정하기 위해 30km/h 이상의 속도로 몇 분 동안 차를 운전하십시오.

참고: 경우에 따라 타이어 공기압과 관련된 경고 기호 또는 메시지를 지우기 위해 15분 이상 운전해야 할 수도 있습니다. 경고가 지속되면 타이어를 식히고 위의 단계를 반복하십시오.

바람 빠진 타이어:

운전 중 타이어 펑크가 났을 경우 일시적으로 펑크를 봉인하고 타이어 이동성 키트를 사용하여 타이어에 다시 공기를 주입할 수 있습니다. 이 키트는 일반적으로 차량의 수하물 보관함
안감 아래에 보관됩니다.

타이어 이동성 키트 사용 지침:

    차량의 테일게이트 또는 트렁크를 엽니다.
    타이어 모빌리

## 7. Stream 답변 생성

모델이 컨텍스트를 이해하고 관련 답변을 생성하는 데 시간이 걸리므로 몇 초 동안 응답을 기다려야 하므로 사용자 경험이 좋지 않을 수 있습니다.

Bedrock은 또한 모델이 토큰을 생성할 때 서비스가 출력을 생성하는 스트리밍 기능을 지원합니다. 다음은 그렇게 할 수 있는 방법의 예입니다.

In [14]:
from IPython.display import display_markdown,Markdown,clear_output

In [15]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['completion']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

 Audi A8의 펑크 난 타이어를 고치는 방법은 다음과 같습니다:

1. 차량의 테일게이트나 트렁크를 엽니다.

2. 러기지 영역의 라이닝을 들어올려 타이어 모빌리티 키트에 접근합니다. 

3. 타이어 이동성 키트와 함께 제공된 지침에 따라 펑크 난 타이어의 구멍을 밀봉합니다. 

4. 키트를 사용하여 타이어에 다시 공기를 넣습니다.

5. 키트를 다 사용하고 난 뒤, 원래의 위치에 안전하게 다시 보관합니다.

6. 사용한 실란트 병은 Rivesla나 적절한 서비스 센터에 문의하여 폐기 및 교체를 지원받습니다.

7. 펑크난 타이어는 가능한 빨리 교체하거나 전문가에게 수리를 맡기는 것이 좋습니다.

8. 타이어 이동성 키트는 임시 해결책이므로 최대 80km/h 이하의 속도로 10분 이내 또는 8km 이내의 거리만 운전하는 것이 권장됩니다.

## 8. 요약

타이어 교체 방법에 대한 응답이 요약되고 단계별 지침임을 알 수 있습니다. 이 간단한 예는 'RAG' 또는 Augmentation 프로세스를 활용하여 선별된 응답을 다시 생성하는 방법을 보여줍니다.